In [71]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from statistics import mean
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np

In [107]:
def get_model():
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [73]:
dataset_path = 'dataset.csv'
dataset = pd.read_csv(dataset_path)

In [74]:
dataset

,Cografya,Yerlesim,AracYogunlugu,YayaYogunlugu,VeriHizi,Guvenilirlik,Gecikme,Kapsama,Standart
0,5,6,6,7,9,6,4,6,1
1,6,2,6,1,3,2,6,7,0
2,5,1,6,9,3,6,6,5,1
3,8,2,3,5,7,5,7,9,0
4,5,7,4,8,5,9,5,6,1
...,...,...,...,...,...,...,...,...,...
9995,4,6,4,4,1,9,9,9,0
9996,1,10,6,10,6,4,8,5,0
9997,4,4,2,6,4,4,9,5,0
9998,4,9,7,2,5,4,2,7,0


In [75]:
x = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1:]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

In [108]:
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, random_state=42, shuffle=True)
batch_sizes = [2**i for i in range(7,11)]
results = []

for batch_size in batch_sizes:
    accuracy_list = []
    for train_cv_idx, val_idx in skf.split(x_train, y_train):
        x_train_cv, x_val = x_train.iloc[train_cv_idx], x_train.iloc[val_idx]
        y_train_cv, y_val = y_train.iloc[train_cv_idx].values.ravel(), y_train.iloc[val_idx].values.ravel()
        model = get_model()
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(x_train_cv, y_train_cv, validation_data=(x_val, y_val),
                  epochs=100, batch_size=batch_size, verbose=0, 
                  callbacks=[EarlyStopping(monitor='val_accuracy', patience=15, mode='max', 
                                           verbose=0, restore_best_weights=True)])
        loss, accuracy = model.evaluate(x_val, y_val, verbose=0)
        accuracy_list.append(accuracy)
    print('Validation accuracy for batch size =\t', batch_size, '\t:', mean(accuracy_list))
    results.append(mean(accuracy_list))
    
max_val = max(results)
max_idx = results.index(max_val)

print('*' * 50)
print('The best validation accuracy was obtained with batch size =', batch_sizes[max_idx], 'as', max_val)

Validation accuracy for batch size =	 128 	: 0.8177500009536743
Validation accuracy for batch size =	 256 	: 0.8142499923706055
Validation accuracy for batch size =	 512 	: 0.8091249942779541
Validation accuracy for batch size =	 1024 	: 0.8024999856948852
**************************************************
The best validation accuracy was obtained with batch size = 128 as 0.8177500009536743


In [109]:
model = get_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train.values.ravel(), epochs=100,
          batch_size=batch_sizes[max_idx], verbose=0, 
          callbacks=[EarlyStopping(monitor='accuracy', patience=15, mode='max', 
                                   verbose=0, restore_best_weights=True)])
pred = model.predict(x_test)
pred = np.where(pred > 0.5, 1, 0)

cm = confusion_matrix(y_test, pred)
print(cm)

[[1050  141]
 [ 221  588]]


In [110]:
print('Test accuracy:', accuracy_score(y_test, pred))

Test accuracy: 0.819


In [111]:
label_names = ['3GPP', 'IEEE']
report = classification_report(y_test, pred, target_names=label_names)
print(report)

              precision    recall  f1-score   support

        3GPP       0.83      0.88      0.85      1191
        IEEE       0.81      0.73      0.76       809

    accuracy                           0.82      2000
   macro avg       0.82      0.80      0.81      2000
weighted avg       0.82      0.82      0.82      2000



In [112]:
model.save('ann.h5')